In [ ]:
import numpy as np
import pandas as pd
import linearsolve as ls
import matplotlib.pyplot as plt
plt.style.use('classic')
plt.rcParams['figure.facecolor'] = 'white'

# Homework 8

**Instructions:** Complete the notebook below. Download the completed notebook in HTML format. Upload assignment using Canvas.

### Exercise: Changing $\beta$ in Prescott's Real Business Cycle Model

Recall that the equilibrium conditions for Prescott's RBC model are:

\begin{align}
\frac{1}{C_t} & = \beta E_t \left[\frac{\alpha A_{t+1}K_{t+1}^{\alpha-1}L_{t+1}^{1-\alpha} +1-\delta }{C_{t+1}}\right]\\
\frac{\varphi}{1-L_t} & = \frac{(1-\alpha)A_tK_t^{\alpha}L_t^{-\alpha}}{C_t} \\
Y_t & = A_t K_t^{\alpha}L_t^{1-\alpha}\\
K_{t+1} & = I_t + (1-\delta) K_t\\
Y_t & = C_t + I_t\\
\log A_{t+1} & = \rho \log A_t + \epsilon_{t+1}
\end{align}

where $\epsilon_{t+1} \sim \mathcal{N}(0,\sigma^2)$. 

The objective is use `linearsolve` to simulate impulse responses to a TFP shock for $\beta = 0.96,0.97,0.98,0.99$. Other parameter values are given in the table below:

| $$\sigma$$ | $$\rho$$  | $$\varphi$$ | $$\alpha$$ | $$\delta $$ |
|------------|-----------|-------------|------------|-------------|
| 0.006      | 0.75      | 1.7317      | 0.35       |  0.025      |


### Model Preparation

As usual, we recast the model in the form required for `linearsolve`. Write the model with all variables moved to the right-hand side of the equations and dropping the expecations operator $E_t$ and the exogenous shock $\epsilon_{t+1}$:

\begin{align}
0 & = \beta\left[\frac{\alpha A_{t+1}K_{t+1}^{\alpha-1}L_{t+1}^{1-\alpha} +1-\delta }{C_{t+1}}\right] - \frac{1}{C_t}\\
0 & = \frac{(1-\alpha)A_tK_t^{\alpha}L_t^{-\alpha}}{C_t} - \frac{\varphi}{1-L_t}\\
0 & = A_t K_t^{\alpha}L_t^{1-\alpha} - Y_t\\
0 & = I_t + (1-\delta) K_t - K_{t+1}\\
0 & = C_t + I_t - Y_t\\
0 & = \rho \log A_t - \log A_{t+1}
\end{align}

Remember, capital and TFP are called *state variables* because they're $t+1$ values are predetermined. Output, consumption, and investment are called a *costate* or *control* variables. Note that the model as 5 equations in 5 endogenous variables.

### Initialization, Approximation, and Solution

The next several cells initialize the model in `linearsolve` and then approximate and solve it.

In [ ]:
# Create a variable called 'parameters' that stores the model parameter values in a Pandas Series.


# Print the model's parameters


In [ ]:
# Create variable called `exo_states` that stores the names of each exogenous state variable.


# Create variable called `endo_states` that stores the names of each endogenous state variable.


# Create variable called `costates` that stores the names of the non-predetermined variables.


In [ ]:
# Define a function that evaluates the equilibrium conditions of the model solved for zero


    # Parameters


    # Current variables


    # Forward variables


    # Euler equation


    # Labor-labor choise


    # Production function


    # Capital evolution


    # Market clearing


    # Exogenous tfp


    # Stack equilibrium conditions into a numpy array




Next, initialize the model using `ls.model` which takes the following required arguments:

* `equations`
* `n_states`
* `var_names`
* `shock_names`
* `parameters`

In [ ]:
# Initialize the model into a variable named 'rbc_model'




### Simulation and Plotting

Create a $2\times 2$ grid of plots containing the impulse responses of TFP, output, labor, and consumption to a one percent shock to TFP for each of the values for $\beta$: 0.96,0.97,0.98,0.99. Your figure should have a legend that clearly indicates which curves were produced from 


Here are the steps that that you should take:

1. Initilize figure and axes for plotting.
2. Iterate over each desired value for $\beta$.
    1. Set `rbc_model.parameters['beta']` equal to current value of $\beta$.
    2. Use `rbc_model.compute_ss()` to compute the steady state with `guess` equal to `[1,4,1,1,1,0.5]`.
    3. Use `rbc_model.approximate_and_solve()` to approximate and solve the model with the current value of $\beta$.
    4. Use `rbc_model.impulse()` to compute the **31** period impulse response to a 0.01 unit shock to TFP in period 5.
    5. Add the computed impulse responses to the axes.
    
Be sure to add a legend somewhere in your figure so that it's clear which impulse response lines were determined by which value of $\beta$.

**Questions**

1. Describe in your own words how increasing $\beta$ from 0.96 to 0.99 affects the impulse response of consumption to a TFP shock.
2. Explain in your own words the intuition for *why* your observation in the previous question makes sense.
3. Describe in your own words how increasing $\beta$ from 0.96 to 0.99 affects the impulse response of labor to a TFP shock.
4. Explain in your own words the intuition for *why* your observation in the previous question makes sense.

**Answers**

1.  

2.  

1.  

2.  

## Exercise: Monetary policy Regimes

In August 6, 1979, Paul Volcker began his first of two terms as chair of the Board of Governors of the Federal Reserve System. In the two decades prior to Volcker's appointment, inflation in the US had been acccelerating rapidly, as the graph produced by the code in the next cell shows:

In [ ]:
# Import GDP deflator data from FRED. CELL PROVIDED
tables = pd.read_html("https://fred.stlouisfed.org/data/GDPDEF", index_col=0)
deflator = tables[1]['VALUE']
deflator.index = pd.to_datetime(deflator.index, format="%Y-%m-%d", errors="coerce")

# Compute the inflation rate
inflation = (deflator - deflator.shift(4))/deflator.shift(4)*100

# Plot
inflation.plot(title='US GDP Deflator Inflation',ylabel='Percent',grid=True);

The decline and stabilization of inflation in the US starting in the early 1980s is widely attributed to Volcker's leadership. As chair, he pushed the FOMC to aggressively pursue tight monetary policy that led to a sharp contraction in the rate of money growth and an increase in the federal fund rate.

In light of this story, many economists argue that Volcker was the start of a new monetary policy *regime*. Before Volcker, the Fed pursued a looser monetary policy that allowed inflation to accelerate and starting with Volcker, the Fed has pursued a tighter monetary policy that is more aggressive to managing inflation.

Here you will test this proposition by estimating a monetary policy rule for the US on data before Volcker's arrival at the Fed and after. Basically, you will replicate the estimation at the end of class 16 for pre- and post-Volcker data.

### Preliminaries

The next block of code imports federal funds rate and output gap data from FRED and returs the data as a DataFrame called `df`.

In [ ]:
# Initial an empty DataFrame that will store data. CELL PROVIDED
df = pd.DataFrame()

# Import federal funds rate data from FRED. Use arguments: sep='\s+',skiprows=62,index_col=0,parse_dates=True
tables = pd.read_html("https://fred.stlouisfed.org/data/FEDFUNDS", index_col=0)
fed_funds = tables[1]['VALUE']
fed_funds.index = pd.to_datetime(fed_funds.index, format="%Y-%m-%d", errors="coerce")

# Use .resample('QS').mean() method of 'fed_funds' to convert the fed funds data from monthdy to quarterly
fed_funds = fed_funds.resample('QS').mean()

# Import real GDP data from FRED
tables = pd.read_html("https://fred.stlouisfed.org/data/GDPC1", index_col=0)
gdp_actual = tables[1]['VALUE']
gdp_actual.index = pd.to_datetime(gdp_actual.index, format="%Y-%m-%d", errors="coerce")

# Import potential real GDP data from FRED
tables = pd.read_html("https://fred.stlouisfed.org/data/GDPPOT", index_col=0)
gdp_potential = tables[1]['VALUE']
gdp_potential.index = pd.to_datetime(gdp_potential.index, format="%Y-%m-%d", errors="coerce")

# Create variable 'df' that is a DataFrame storing fed funds, inflation, actual and potential real GDP
df = pd.DataFrame({
    'fed_funds':fed_funds,
    'inflation':inflation,
    'gdp_actual':gdp_actual,
    'gdp_potential':gdp_potential
})

# Drop missing values from 'df'
df = df.dropna()

### The Output Gap

The output gap is measured as the percent difference of real GDP from the CBO's estimate of the potential real GDP:

\begin{align}
\text{Output gap} & = \left(\frac{\text{Real GDP}-\text{Real potential GDP}}{\text{Real potential GDP}}\right)\cdot 100
\end{align}

Real GDP has FRED series ID `GDPC1` and is available here: https://fred.stlouisfed.org/series/GDPC1. Notice that there are 17 lines of text *before* the line starting with "DATE".

In [ ]:
# Construct an output gap column


### OLS Regressions

The rule to be estimated is the same considered in Class 16:

\begin{align}
\hat{i}_t & = \bar{\imath} + \phi_{\pi}\pi_t + \phi_{y} y_t + \epsilon_t
\end{align}

where $\pi_t$ is the percent change in the GDP deflator over the previous year and $y_t$ is the output gap measured as the percent difference of real GDP from the CBO's estimate of the potential real GDP. $\phi_{\pi}$ is the weight that the FOMC places on inflation in the rule and $\phi_{y}$ is the weight that the central bank places on the output gap. $\epsilon_t$ is the residual of the regression. 

In [ ]:
# Import statsmodels.api as sm


For a Pandas DataFrame or Series with a DateTime index, you can select a subinterval of the dates using data strings in `.ilo[]`. For example, 

    df.loc[:'1960']
    
will return all data through the end of 1960. And:

    df.loc['08-2000':]
    
will return all data from and after August 2000.

#### Pre-Volcker Era

Estimate the monetary policy rule for dates through July 1979.

In [ ]:
# Create variable 'X' with columns inflation, output and a constant


# Create variable 'Y' equal to the federal funds rate


In [ ]:
# Initialize OLS model


# Fit OLS model


# Print regression results


#### Post-Volcker Era

Estimate the monetary policy rule for dates starting October 1979.

In [ ]:
# Create variable 'X' with columns inflation, output and a constant


# Create variable 'Y' equal to the federal funds rate


In [ ]:
# Initialize OLS model


# Fit OLS model


# Print regression results


**Questions**

1. Compare the results from the two regressions. For which monetary policy era is the estimated coefficient on inflation greater?
2. How do your results help to explain the high and variable inflation of the 1960s and 1970s and the lower and more stable inflation of the 1980s and after?

**Answers**

1.  

2.  